#1. Importações

In [ ]:
!pip install pandas nltk scikit-learn joblib lime

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import joblib
from lime.lime_text import LimeTextExplainer

#2. Limpeza de dados


In [ ]:
import nltk
nltk.download('stopwords')

def refine_lyric(lyric_content):
    if isinstance(lyric_content, list):
        lyric_content = lyric_content[0]
    lyric_content = re.sub(r'[^\w\s]', '', lyric_content)
    lyric_content = lyric_content.lower()
    lyric_content = lyric_content.replace('\n', ' ')
    return lyric_content

#3. Predict Class


In [ ]:
class SongClassifier:
    def __init__(self, model_file):
        self.predictive_model = joblib.load(model_file)

    def classify_song(self, lyrics):
        processed_lyrics = refine_lyric(lyrics)
        return self.predictive_model.predict([processed_lyrics])[0]

    def probability_distribution(self, multiple_lyrics):
        processed_texts = [refine_lyric(lyric) for lyric in multiple_lyrics]
        return self.predictive_model.predict_proba(processed_texts)

#4. Explain Class

In [ ]:
class GenreAnalyser:
    def __init__(self, model):
        self.song_model = model
        self.genres_list = model.predictive_model.classes_
        self.explanation_engine = LimeTextExplainer(class_names=self.genres_list)

    def analyse_genre(self, song_lyrics):
        lyrics_for_analysis = refine_lyric(song_lyrics)
        explanation = self.explanation_engine.explain_instance(lyrics_for_analysis, self.song_model.probability_distribution, num_features=6, top_labels=1)
        explanation.show_in_notebook(text=True)

#5. Teste

In [ ]:
song_analyser = SongClassifier ('') # Insira o nome do arquivo do modelo aqui
genre_inspector = GenreAnalyser(song_analyser)

lyrics_input = input("Digite a letra da música: ")

print("Gênero previsto:", song_analyser.classify_song(lyrics_input))

genre_inspector.analyse_genre(lyrics_input)